## Imports

In [4]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
from tabula.io import read_pdf
import re

## Import data from PDF

In [6]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

## Pandas Manipulation

In [7]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [8]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

## Geocoder Setup and Execution

In [10]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [11]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [12]:
df['loc']

1     (16901 Collins Ave #5303, Sunny Isles Beach, F...
2     (360 Ocean Dr #603s, Key Biscayne, FL 33149, U...
3     (10225 Collins Ave, Bal Harbour, FL 33154, USA...
4     (1670 Lincoln Ct #6a, Miami Beach, FL 33139, U...
5     (1040 Biscayne Blvd APT 4403, Miami, FL 33132,...
6     (8701 Collins Ave #1205, Miami Beach, FL 33154...
7     (101 20th St #1907, Miami Beach, FL 33139, USA...
8     (16047 Collins Ave APT 702, Sunny Isles Beach,...
9     (17001 Collins Ave APT 3705, Sunny Isles Beach...
10    (Asia, 900 Brickell Key Blvd APT 1604, Miami, ...
Name: loc, dtype: object

In [13]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [14]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Jade Signature\r16901 Collins Ave 5303\rSunny ...,2/9/2018,6/13/2022,1566,"$10,400,000.00",10213,"$1,649.48",Rita Collins,Fortune International Realty,Fernando Alves,Chares Rutenberg Realty,Jade Signature 16901 Collins Ave 5303 Sunny Is...,Jade Signature,16901 Collins Ave 5303 Sunny Isles Beach,"(16901 Collins Ave #5303, Sunny Isles Beach, F...","(25.9327557, -80.1210054, 0.0)",25.932756,-80.121005,0.0
2,Oceana Key Biscayne\r360 Ocean Dr 603S\rKey Bi...,4/11/2022,6/13/2022,7,"$7,400,000.00",3013,"$2,456.02",Giulietta Ulloa,BHHS EWM Realty,Giulietta Ulloa,BHHS EWM Realty,Oceana Key Biscayne 360 Ocean Dr 603S Key Bisc...,Oceana Key Biscayne,360 Ocean Dr 603S Key Biscayne,"(360 Ocean Dr #603s, Key Biscayne, FL 33149, U...","(25.6942877, -80.1578218, 0.0)",25.694288,-80.157822,0.0
3,Bellini\r10225 Collins Ave 502/504\rBal Harbour,6/30/2021,6/14/2022,289,"$5,850,000.00",6738,$868.21,Bryan Sereny PA,Douglas Elliman,Sharona Litwin,Realty 24/7 Co,Bellini 10225 Collins Ave 502/504 Bal Harbour,Bellini,10225 Collins Ave 502/504 Bal Harbour,"(10225 Collins Ave, Bal Harbour, FL 33154, USA...","(25.8965503, -80.1230557, 0.0)",25.896550,-80.123056,0.0
4,Bonne Vie\r1670 Lincoln Ct 6A\rMiami Beach,5/12/2022,6/14/2022,15,"$5,000,000.00",757,"$6,605.02",Marco Rangel Romero,Avanti Way Realty LLC,Jeffrey Rodriguez,ASLO Rfeal Estate,Bonne Vie 1670 Lincoln Ct 6A Miami Beach,Bonne Vie,1670 Lincoln Ct 6A Miami Beach,"(1670 Lincoln Ct #6a, Miami Beach, FL 33139, U...","(25.7912512, -80.1444502, 0.0)",25.791251,-80.144450,0.0
5,Ten Museum Park\r1040 Biscayne Blvd 4403\rMiami,2/10/2022,6/13/2022,62,"$3,850,000.00",4327,$889.76,Scott Klein PA,Insignia International Properties,Alfredo Caprille,Coldwell Banker Realty,Ten Museum Park 1040 Biscayne Blvd 4403 Miami,Ten Museum Park,1040 Biscayne Blvd 4403 Miami,"(1040 Biscayne Blvd APT 4403, Miami, FL 33132,...","(25.7846753, -80.19018, 0.0)",25.784675,-80.190180,0.0
6,Eighty Seven Park\r8701 Collins Ave 1205\rMiam...,Not Available,6/16/2022,Not Available,"$3,850,000.00",1522,"$2,529.57",Comp Sale,Douglas Elliman,Oliver Lloyd,Dezer Platinum Realty LLC,Eighty Seven Park 8701 Collins Ave 1205 Miami ...,Eighty Seven Park,8701 Collins Ave 1205 Miami Beach,"(8701 Collins Ave #1205, Miami Beach, FL 33154...","(25.8720993, -80.1218779, 0.0)",25.872099,-80.121878,0.0
7,Setai\r101 20 St 1907\rMiami Beach,11/23/2021,6/13/2022,139,"$3,800,000.00",1279,"$2,971.07",Giorgio Vecchi,Douglas Elliman,Perel Weiss,M. Kotler Realty Inc,Setai 101 20 St 1907 Miami Beach,Setai,101 20 St 1907 Miami Beach,"(101 20th St #1907, Miami Beach, FL 33139, USA...","(25.7956082, -80.1281603, 0.0)",25.795608,-80.128160,0.0
8,Turnberry Ocean Colony\r16047 Collins Ave 702\...,1/24/2022,6/15/2022,78,"$2,384,000.00",2680,"$1,061.92",Lana Bell,Douglas Elliman,Dominique Gagnon,One Sotheby's International,Turnberry Ocean Colony 16047 Collins Ave 702 S...,Turnberry Ocean Colony,16047 Collins Ave 702 Sunny Isles Beach,"(16047 Collins Ave APT 702, Sunny Isles Beach,...","(25.9252659, -80.1216882, 0.0)",25.925266,-80.121688,0.0
9,Jade Beach\r17001 Collins Ave 3705\rSunny Isle...,4/4/2022,6/17/2022,57,"$2,260,000.00",1927,"$1,172.81",Felipe Quintero,Nexxos Realty LLC,Rodrigo Nahabetian,Grand Realty of America Corp.,Jade Beach 17001 Collins Ave 3705 Sunny Isles ...,Jade Beach,17001 Collins Ave 3705 Sunny Isles Beach,"(17001 Collins Ave APT 3705, Sunny Isles Beach...","(25.9335244, -80.1211641, 0.0)",25.933524,-80.121164,0.0
10,A0sia Condo\r900 Brickell Key Blvd 1604\rMiami,3/22/2022,6/14/2022,31,"$2,250,000.00",2627,$856.49,Jorge Salazar,Douglas Elliman,Melanie Rutkin,Miami Waterfront Realty,A0sia Condo 900 Brickell Key Blvd 1604 Miami,A,0sia Condo 900 Brickell Key Blvd 1604 Miami,"(Asia, 900 Brickell Key Blvd APT 1604, Miami, ...","(25.7690295, -80.185697

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction Section
To be used if a name is off. Format for making corrections is as follows:

`df.at[index_num,'column_name']=('what you want inserted')`

In [15]:
df.at[10,'building_name']=('Asia')
# df.at[2,'address_only']=('1 Collins Ave 307 Miami Beach')

In [18]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('June 12th - June 18th')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [19]:
m.save('index.html')